In [36]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

from dataset.countdown_dataloader import Countdown
from dataset.countdown_utils import ( gen_dataset, compute_metrics )

In [37]:
# device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [38]:
# Create and save a tiny dataset with 5 samples
dataset_json_path = "simpler_countdown_data.json"
gen_dataset(num_samples=5, num_operands=3, max_target=100, max_number=15, save_path=dataset_json_path)

# Load the dataset
dataset = Countdown(dataset_json_path)

In [39]:
def generate_output(model, tokenizer, target: int, numbers: list) -> str:
    """
    Given a target integer and a list of numbers,
    prompt the model to create an equation that equals the target.
    """
    prompt = f"Using the numbers {numbers}, create an equation that equals {target}. Box your answer."
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v for k, v in inputs.items()}
    outputs = model.generate(**inputs, max_new_tokens=1000)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [40]:
model_name = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

In [41]:
model.eval()
for sample in dataset:
    target = sample["target"]
    numbers = sample["numbers"]
    output = generate_output(model, tokenizer, target, numbers)
    print(f"Target: {target}, Numbers: {numbers}, Equation: {output}")
    # Compute metrics
    metrics = compute_metrics(output, sample)
    print(f"Metrics: {metrics}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Target: 5, Numbers: [1, 2, 4], Equation: Using the numbers [1, 2, 4], create an equation that equals 5. Box your answer. To create an equation that equals 5 using the numbers [1, 2, 4], we can use the following steps:

1. Start with the number 1.
2. Multiply it by 2.
3. Add 4 to the result from step 2.
4. Add 1 to the result from step 3.
5. Add 2 to the result from step 4.
6. Add 4 to the result from step 5.
7. Add 1 to the result from step 6.
8. Add 2 to the result from step 7.
9. Add 4 to the result from step 8.
10. Add 1 to the result from step 9.
11. Add 2 to the result from step 10.
12. Add 4 to the result from step 11.
13. Add 1 to the result from step 12.
14. Add 2 to the result from step 13.
15. Add 4 to the result from step 14.
16. Add 1 to the result from step 15.
17. Add 2 to the result from step 16.
18. Add 4 to the result from step 17.
19. Add 1 to the result from step 18.
20. Add 2 to the result from step 19.
21. Add 4 to the result from step 20.
22. Add 1 to the result f

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Target: 46, Numbers: [14, 6, 10], Equation: Using the numbers [14, 6, 10], create an equation that equals 46. Box your answer. To create an equation that equals 46 using the numbers 14, 6, and 10, we need to find a combination of these numbers that, when added together, equal 46. Let's go through the steps:

1. Start with the largest number, 10.
2. Subtract 10 from 46 to get 36.
3. Now, we need to find a combination of 6 and 14 that add up to 36. Let's try different combinations:

   - If we use 6 and 14, we get \(6 + 14 = 20\).
   - If we use 10 and 14, we get \(10 + 14 = 24\).
   - If we use 10 and 6, we get \(10 + 6 = 16\).
   - If we use 14 and 6, we get \(14 + 6 = 20\).

   None of these combinations add up to 36. Let's try another combination:

   - If we use 10 and 14, we get \(10 + 14 = 24\).
   - If we use 10 and 6, we get \(10 + 6 = 16\).
   - If we use 14 and 6, we get \(14 + 6 = 20\).

   None of these combinations add up to 36. Let's try another combination:

   - If we us

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Target: 88, Numbers: [11, 2, 10], Equation: Using the numbers [11, 2, 10], create an equation that equals 88. Box your answer. To create an equation that equals 88 using the numbers 11, 2, and 10, we need to find a combination of these numbers that, when added together, equal 88. Let's go through the steps to find the solution:

1. Start by considering the largest number, 10, and see if we can use it to form a sum that equals 88.
2. Subtract 10 from 88 to see if we can get 88 by using the remaining numbers.

Let's try this:

\[ 88 - 10 = 78 \]

Now, we need to find a combination of the remaining numbers (11 and 2) that add up to 78. Let's try different combinations:

- If we use 11 and 2, we get \( 11 + 2 = 13 \), which is not 78.
- If we use 11 and 10, we get \( 11 + 10 = 21 \), which is not 78.
- If we use 10 and 11, we get \( 10 + 11 = 21 \), which is not 78.
- If we use 10 and 2, we get \( 10 + 2 = 12 \), which is not 78.
- If we use 11 and 10, we get \( 11 + 10 = 21 \), which is n

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Target: 32, Numbers: [3, 8, 7], Equation: Using the numbers [3, 8, 7], create an equation that equals 32. Box your answer. To create an equation that equals 32 using the numbers 3, 8, and 7, we need to find a combination of these numbers that, when used in an arithmetic operation, result in 32. Let's explore different combinations:

1. **Using the numbers 3, 8, and 7:**
   - We can try different combinations of these numbers to see if they add up to 32.
   - One possible combination is \(3 + 8 + 7 = 18\), but this does not equal 32.
   - Another possible combination is \(3 + 7 + 8 = 18\), but this also does not equal 32.
   - Another possible combination is \(8 + 3 + 7 = 18\), but this also does not equal 32.
   - Another possible combination is \(8 + 7 + 3 = 18\), but this also does not equal 32.
   - Another possible combination is \(7 + 3 + 8 = 18\), but this also does not equal 32.
   - Another possible combination is \(7 + 8 + 3 = 18\), but this also does not equal 32.
   - Anothe